<a href="https://colab.research.google.com/github/Ayomidemm/Ayomidemm/blob/main/CAPSTONE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the COVID-19 dataset
data = pd.read_csv("covid_data.csv")

# Handle missing values
data.fillna(method='ffill', inplace=True)

# Convert date column to datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Feature engineering
data['New Cases'] = data['Confirmed'].diff()
data['New Deaths'] = data['Deaths'].diff()
data['New Recoveries'] = data['Recovered'].diff()

# Create a new dataset for time series forecasting
ts_data = data[['Date', 'Confirmed']]
ts_data.set_index('Date', inplace=True)

# Visualize the trend of confirmed cases
plt.figure(figsize=(12, 6))
plt.plot(ts_data['Confirmed'])
plt.xlabel('Date')
plt.ylabel('Confirmed Cases')
plt.title('Trend of Confirmed Cases')
plt.show()

# Calculate daily growth rate
ts_data['Daily Growth Rate'] = ts_data['Confirmed'].pct_change()

# Visualize the daily growth rate
plt.figure(figsize=(12, 6))
plt.plot(ts_data['Daily Growth Rate'])
plt.xlabel('Date')
plt.ylabel('Daily Growth Rate')
plt.title('Daily Growth Rate of Confirmed Cases')
plt.show()


#TIME SERIES

from fbprophet import Prophet

# Prepare data for Prophet
df = ts_data.reset_index()
df.columns = ['ds', 'y']

# Create and fit the Prophet model
model = Prophet()
model.fit(df)

# Make future predictions
future = model.make_future_dataframe(periods=30)
forecast = model.predict(future)

# Visualize the forecast
fig1 = model.plot(forecast)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Assuming a 'Severity' column exists in the data indicating case severity
X = data[['Age', 'Gender', 'Comorbidities']]
y = data['Severity']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and train the logistic regression model
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(conf_matrix)